In [1]:
import numpy as np
import pandas as pd
import requests
import sys
import regex as re

In [2]:
BAF = pd.read_csv("data/BAF.csv")
mediator = pd.read_csv("data/mediator.csv")
P300CBP = pd.read_csv("data/P300CBP.csv")
SAGA = pd.read_csv("data/SAGA.csv")
TFIID = pd.read_csv("data/TFIID.csv")
TIP60 = pd.read_csv("data/TIP60.csv")
others = pd.read_csv("data/others.csv")

combined = pd.concat([BAF, mediator, P300CBP, SAGA, TFIID, TIP60, others]).drop_duplicates()

In [3]:
mappedcofactors = pd.read_csv("data/cofactors_mapped.tsv", delimiter="\t")

#Check there is a reviewed version of each Uniprot entry, remove unreviewed versions
if all(mappedcofactors["From"].unique() == mappedcofactors[mappedcofactors["Reviewed"] == "reviewed"]["From"].unique()):
    mappedcofactors = mappedcofactors[mappedcofactors["Reviewed"] == "reviewed"]

#Check all of the cofactors in combined list are included in mapped cofactors (due to name duplication)
cofactornames = " ".join(mappedcofactors["Gene Names"].to_list()).split(" ")
print("All cofactors included? ", all([True for i in combined["Gene Name"] if i in cofactornames]))

#Clean ; from Bgee column
mappedcofactors["Bgee"] = mappedcofactors["Bgee"].apply(lambda x: x.replace(";", "") if type(x) == str else x)
mappedcofactors["GeneID"] = mappedcofactors["GeneID"].apply(lambda x: x.replace(";", "") if type(x) == str else x)

mappedcofactors = mappedcofactors.rename({"From": "Gene Name", "Entry": "UniprotID"}, axis = 1).drop(["Reviewed"], axis = 1)

mappedcofactors.head()

All cofactors included?  True


,Gene Name,UniprotID,Entry Name,Protein names,Gene Names,Sequence,Bgee,Ensembl,GeneID,PDB
0,BAF180,Q86U86,PB1_HUMAN,Protein polybromo-1 (hPB1) (BRG1-associated fa...,PBRM1 BAF180 PB1,MGSKRRRATSPSSSVSGDFDDGHHSVSTPGPSRKRRRLSNLPTVDP...,ENSG00000163939,ENST00000296302.11 [Q86U86-1];ENST00000337303....,55193,2KTB;3G0J;3HMF;3IU5;3IU6;3K2J;3LJW;3MB4;3TLP;4...
1,BCL11A,Q9H165,BC11A_HUMAN,B-cell lymphoma/leukemia 11A (BCL-11A) (B-cell...,BCL11A CTIP1 EVI9 KIAA1809 ZNF856,MSRRKQGKPQHLSKREFSPEPLEAILTDDEPDHGPLGAPEGDHDLL...,ENSG00000119866,ENST00000335712.11 [Q9H165-6];ENST00000356842....,53335,5VTB;6KI6;6U9Q;8DTN;8DTU;8THO;8TLO;9BV0;
20,BCL11B,Q9C0K0,BC11B_HUMAN,B-cell lymphoma/leukemia 11B (BCL-11B) (B-cell...,BCL11B CTIP2 RIT1,MSRRKQGNPQHLSQRELITPEADHVEAAILEEDEGLEIEEPSGLGL...,ENSG00000127152,ENST00000345514.2 [Q9C0K0-2];ENST00000357195.8...,64919,NaN
24,BAF60A,Q96GM5,SMRD1_HUMAN,SWI/SNF-related matrix-associated actin-depend...,SMARCD1 BAF60A,MAARAGFQSVAPSGGAGASGGAGAAAALGPGGTPGPPVRMGPAPGQ...,ENSG00000066117,ENST00000381513.8 [Q96GM5-2];ENST00000394963.9...,6602,6LTH;6LTJ;7VDV;7Y8R;
25,BAF60B,Q92925,SMRD2_HUMAN,SWI/SNF-related matrix-associated actin-depend...,SMARCD2 BAF60B PRO2451,MSGRGAGGFPLPPLSPGGGAVAAALGAPPPPAGPGMLPGPALRGPG...,ENSG00000108604,ENST00000323347.14 [Q92925-3];ENST00000448276....,6603,NaN


In [4]:
data = combined.merge(mappedcofactors, left_on= "Gene Name", right_on="Gene Name")
data.to_csv("helperfiles/cofactors_mapped_combined.csv")
data.head()

,Gene Name,Complex,Subcomplex or Module,Own-complex paralog,Other-complex Paralogues,Notes,Source,UniprotID,Entry Name,Protein names,Gene Names,Sequence,Bgee,Ensembl,GeneID,PDB
0,BAF180,BAF,esBAF,NaN,NaN,NaN,Alfert et al. Epigenetics & Chromatin (2019),Q86U86,PB1_HUMAN,Protein polybromo-1 (hPB1) (BRG1-associated fa...,PBRM1 BAF180 PB1,MGSKRRRATSPSSSVSGDFDDGHHSVSTPGPSRKRRRLSNLPTVDP...,ENSG00000163939,ENST00000296302.11 [Q86U86-1];ENST00000337303....,55193,2KTB;3G0J;3HMF;3IU5;3IU6;3K2J;3LJW;3MB4;3TLP;4...
1,BAF180,BAF,npBAF,NaN,NaN,NaN,Alfert et al. Epigenetics & Chromatin (2019),Q86U86,PB1_HUMAN,Protein polybromo-1 (hPB1) (BRG1-associated fa...,PBRM1 BAF180 PB1,MGSKRRRATSPSSSVSGDFDDGHHSVSTPGPSRKRRRLSNLPTVDP...,ENSG00000163939,ENST00000296302.11 [Q86U86-1];ENST00000337303....,55193,2KTB;3G0J;3HMF;3IU5;3IU6;3K2J;3LJW;3MB4;3TLP;4...
2,BAF180,BAF,nBAF,NaN,NaN,NaN,Alfert et al. Epigenetics & Chromatin (2019),Q86U86,PB1_HUMAN,Protein polybromo-1 (hPB1) (BRG1-associated fa...,PBRM1 BAF180 PB1,MGSKRRRATSPSSSVSGDFDDGHHSVSTPGPSRKRRRLSNLPTVDP...,ENSG00000163939,ENST00000296302.11 [Q86U86-1];ENST00000337303....,55193,2KTB;3G0J;3HMF;3IU5;3IU6;3K2J;3LJW;3MB4;3TLP;4...
3,BCL11A,BAF,esBAF,[BCL11B],NaN,NaN,Alfert et al. Epigenetics & Chromatin (2019),Q9H165,BC11A_HUMAN,B-cell lymphoma/leukemia 11A (BCL-11A) (B-cell...,BCL11A CTIP1 EVI9 KIAA1809 ZNF856,MSRRKQGKPQHLSKREFSPEPLEAILTDDEPDHGPLGAPEGDHDLL...,ENSG00000119866,ENST00000335712.11 [Q9H165-6];ENST00000356842....,53335,5VTB;6KI6;6U9Q;8DTN;8DTU;8THO;8TLO;9BV0;
4,BCL11A,BAF,npBAF,[BCL11B],NaN,NaN,Alfert et al. Epigenetics & Chromatin (2019),Q9H165,BC11A_HUMAN,B-cell lymphoma/leukemia 11A (BCL-11A) (B-cell...,BCL11A CTIP1 EVI9 KIAA1809 ZNF856,MSRRKQGKPQHLSKREFSPEPLEAILTDDEPDHGPLGAPEGDHDLL...,ENSG00000119866,ENST00000335712.11 [Q9H165-6];ENST00000356842....,53335,5VTB;6KI6;6U9Q;8DTN;8DTU;8THO;8TLO;9BV0;


In [5]:
#List of all unique UniprotID, BGee, GeneID, list of gene names
mapper = data[["Bgee", "GeneID", "UniprotID", "Gene Names"]]
mapper = mapper.groupby("UniprotID").agg(lambda x: list(set(x.to_list())))
mapper["Bgee"] = mapper["Bgee"].apply(lambda x: x[0])
mapper["GeneID"] = mapper["GeneID"].apply(lambda x: x[0])
mapper["Gene Names"] = mapper["Gene Names"].apply(lambda x: x[0].split(" "))
mapper

,Bgee,GeneID,Gene Names
UniprotID,,,
A0JLT2,ENSG00000156603,219541,"[MED19, LCMR1]"
O00267,ENSG00000196235,6829,"[SUPT5H, SPT5, SPT5H]"
O14497,ENSG00000117713,8289,"[ARID1A, BAF250, BAF250A, C1orf4, OSA1, SMARCF1]"
O14646,ENSG00000153922,1105,[CHD1]
O14686,ENSG00000167548,8085,"[KMT2D, ALR, MLL2, MLL4]"
...,...,...,...
Q9Y3C7,ENSG00000108590,51003,"[MED31, SOH1, CGI-125]"
Q9Y4A5,ENSG00000196367,8295,"[TRRAP, PAF400]"
Q9Y5B9,ENSG00000092201,11198,"[SUPT16H, FACT140, FACTP140]"


In [7]:
%% skip
#Generated mapper_GTEXcode.csv

def gtex_getgenecode(ensemblid):
    """
    Takes an ensemblID (e.g ENSGxxxxx) and retrieves its GTEX geneID (ENSGxxxxx.xx)
    """
    try:
        server = "https://gtexportal.org/api/v2/reference/gene?"
        ext = "geneId=" + str(ensemblid)

        r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
        if not r.ok:
            r.raise_for_status()
            sys.exit()
        

        decoded = r.json()
        return decoded["data"][0]["gencodeId"]
    
    except Exception as e:
        print(e, ensemblid)
        return np.NaN


def gtex_getgenecodes(mapperDF, export):
    """
    Adds "GTEXGeneCode" column to DF with the GTEX genecode (ENSGxxxx.xx) of the gene
    """
    genecodes = []
    for ensemblID in mapperDF["Bgee"]:
        genecodes.append(gtex_getgenecode(ensemblID))

    mapperDF["GTEXGeneCode"] = genecodes

    if export:
        mapperDF.to_csv("helperfiles/mapper_GTEXcode.csv")

    return mapperDF

gtex_getgenecodes(mapper, True)

list index out of range nan


,Bgee,GeneID,Gene Names,GTEXGeneCode
UniprotID,,,,
A0JLT2,ENSG00000156603,219541,"[MED19, LCMR1]",ENSG00000156603.15
O00267,ENSG00000196235,6829,"[SUPT5H, SPT5, SPT5H]",ENSG00000196235.13
O14497,ENSG00000117713,8289,"[ARID1A, BAF250, BAF250A, C1orf4, OSA1, SMARCF1]",ENSG00000117713.18
O14646,ENSG00000153922,1105,[CHD1],ENSG00000153922.10
O14686,ENSG00000167548,8085,"[KMT2D, ALR, MLL2, MLL4]",ENSG00000167548.14
...,...,...,...,...
Q9Y3C7,ENSG00000108590,51003,"[MED31, SOH1, CGI-125]",ENSG00000108590.10
Q9Y4A5,ENSG00000196367,8295,"[TRRAP, PAF400]",ENSG00000196367.12
Q9Y5B9,ENSG00000092201,11198,"[SUPT16H, FACT140, FACTP140]",ENSG00000092201.9


In [9]:
mapper_gtex = pd.read_csv("helperfiles/mapper_GTEXcode.csv")
mapper_gtex.head()

,UniprotID,Bgee,GeneID,Gene Names,GTEXGeneCode
0,A0JLT2,ENSG00000156603,219541.0,"['MED19', 'LCMR1']",ENSG00000156603.15
1,O00267,ENSG00000196235,6829.0,"['SUPT5H', 'SPT5', 'SPT5H']",ENSG00000196235.13
2,O14497,ENSG00000117713,8289.0,"['ARID1A', 'BAF250', 'BAF250A', 'C1orf4', 'OSA...",ENSG00000117713.18
3,O14646,ENSG00000153922,1105.0,['CHD1'],ENSG00000153922.10
4,O14686,ENSG00000167548,8085.0,"['KMT2D', 'ALR', 'MLL2', 'MLL4']",ENSG00000167548.14


In [18]:
def gtex_getfullexpression(ensemblid):
    """
    Takes an GTEX geneID (ENSGxxxxx.xx) and returns a DataFrame of its expression data
    """
    try:
        server = "https://gtexportal.org/api/v2/expression/geneExpression?"
        ext = "gencodeId=" + str(ensemblid)
        r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
        if not r.ok:
            r.raise_for_status()
            sys.exit()
        
        decoded = r.json()

        return pd.DataFrame(decoded["data"])

    except Exception as e:
        print(ensemblid, e)
        return np.NaN


def fullexpressionDF(ensemblid, save_csv, dirstring):
    """
    Ensemblid: GTEX geneID (ENSGxxxxx.xx) 
    Save_csv (boolean): Saves a CSV file of individual expression 
    dirstring (string): directory to save the csv into
    value (string): 1D df of mean, median, std, count from the expression results to return
    """
    fulldf = gtex_getfullexpression(ensemblid)

    try:
        newdf = fulldf[["tissueSiteDetailId"]].set_index("tissueSiteDetailId")
        newdf["mean"] = fulldf["data"].apply(np.mean).to_list()
        newdf["median"] = fulldf["data"].apply(np.mean).to_list()
        newdf["std"] = fulldf["data"].apply(np.std).to_list()
        newdf["count"] = fulldf["data"].apply(len).to_list()
    except Exception as e:
        newdf = pd.DataFrame({"mean": [], "median": [], "std": [], "count": []})         #No result, return empty df

    if save_csv:
        newdf.to_csv(dirstring + ensemblid + "_expression.csv")

    def select_df(label):
        return newdf[[label]].rename({label: ensemblid}, axis=1)

    return select_df("mean"), select_df("median"), select_df("std"), select_df("count")


In [ ]:
#Step 2: Get List of Ensembl GTEX IDs (ENSGxxxx.xx)

def fullexpressionrun(GTEXIDs, directory, individual, stats):
    """
    Retrieves expression data GTEXIDs.
    GTEXIDs (list-like): list of GTEXIDs
    directory (string): where to save CSV files
    individual (boolean): whether to export expression statistics for each gene
    stats (boolean): whether to export the net stats data of all genes
    """

    meandf = pd.DataFrame({})
    mediandf = pd.DataFrame({})
    stddf = pd.DataFrame({})
    countdf = pd.DataFrame({})

    if individual:
        indiv_dir = directory + "individual_results/"
        !mkdir {indiv_dir}
    else:
        indiv_dir = ""

    for ensemblid in GTEXIDs:
        mean, median, std, count = fullexpressionDF(str(ensemblid), individual, indiv_dir)
        meandf = meandf.merge(mean, right_index=True, left_index=True, how="outer")
        mediandf = mediandf.merge(median, right_index=True, left_index=True, how="outer")
        stddf = stddf.merge(std, right_index=True, left_index=True, how="outer")
        countdf = countdf.merge(count,right_index=True, left_index=True, how="outer")

    if stats:
        meandf.to_csv(directory + "mean_expression.csv")
        mediandf.to_csv(directory + "median_expression.csv")
        stddf.to_csv(directory + "std_expression.csv")
        countdf.to_csv(directory + "counts_expression.csv")

    return meandf, mediandf, stddf, countdf

#### Run on Mapper

mapper_gencode = mapper_gtex.copy()
meandf, mediandf, stddf, countdf = fullexpressionrun(mapper_gencode["GTEXGeneCode"], "cofactor_expression/", True, True)

mkdir: cofactor_expression/individual_results/: File exists


TypeError: can only concatenate str (not "float") to str